In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

### Loading the Model and Testing it

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

In [14]:
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

for model in client.models.list():
    print(model.name)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


models/embedding-gecko-001
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/ge

In [49]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [28]:
message = [SystemMessage(content="You are a helpful assistant."), HumanMessage(content="What is the capital of India?")]

In [34]:
message2 = [SystemMessage(content="You are a helpful assistant and always come up with funny answers."), HumanMessage(content="What is the capital of India?")]

In [29]:
model.invoke(message)

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--12984514-0f98-4b92-be9f-e7619603c617-0', usage_metadata={'input_tokens': 14, 'output_tokens': 9, 'total_tokens': 295, 'input_token_details': {'cache_read': 0}})

In [35]:
model.invoke(message2)

AIMessage(content="Why, it's **New Delhi**, of course!\n\nIt's where all the important decisions are made, and also where you can find some of the best butter chicken on the planet. I suspect the two are related. You can't govern a billion people on an empty stomach, you know", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--90fc5c65-4a30-4992-90c2-92aebe4e1bfc-0', usage_metadata={'input_tokens': 21, 'output_tokens': 62, 'total_tokens': 927, 'input_token_details': {'cache_read': 0}})

### Use Output Parser

In [30]:
from langchain_core.output_parsers import StrOutputParser

In [31]:
parser = StrOutputParser()

In [32]:
response = model.invoke(message)

In [36]:
response2 = model.invoke(message2)

In [33]:
parser.invoke(response)

'The capital of India is **New Delhi**.'

In [37]:
parser.invoke(response2)

"Why, that's New Delhi!\n\nIt's the political capital, but many would argue the *real* capital is wherever you can find the best butter chicken at any given moment. My circuits are still processing the data on that one."

### Chaining with LCEL

In [38]:
chain = model | parser

In [39]:
chain.invoke(message)

'The capital of India is **New Delhi**.'

In [40]:
chain.invoke(message2)

"Ah, an easy one! The capital of India is **New Delhi**.\n\nIt's not to be confused with its older, more chaotic sibling, Old Delhi, which is the capital of street food and finding yourself gloriously lost. One is for filing taxes, the other is for filing your stomach with delicious kebabs. Choose your capital wisely"

### Prompt Templating

In [50]:
from langchain_core.prompts import ChatPromptTemplate

In [51]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "translate the following text to {language}"), ("user", "{text}")]
)

In [52]:
prompt = prompt_template.invoke({"language": "French", "text": "I started learning a new topic today, I want to be a good person"})

In [53]:
parser.invoke(model.invoke(prompt))

'Of course! Here are a few ways to translate that, from the most literal to slightly more nuanced options.\n\n**Most direct translation:**\n"J\'ai commencé à apprendre un nouveau sujet aujourd\'hui, je veux être une bonne personne."\n\n---\n\nHere are some other natural-sounding options:\n\n**Option 2 (Slightly more common phrasing):**\n"Aujourd\'hui, j\'ai commencé à apprendre un nouveau sujet. Je veux devenir une bonne personne."\n*(This version uses "devenir" which means "to become," a very common way to express this idea in French.)*\n\n**Option 3 (A bit more formal/aspirational):**\n"J\'ai commencé à étudier un nouveau domaine aujourd\'hui. J\'aimerais être quelqu\'un de bien."\n*(This uses "étudier" (to study), "domaine" (field/area), "j\'aimerais" (I would like), and "quelqu\'un de bien" (someone good).)*'